In [2]:
# sudo apt install ffmpeg
%pip install -q torch transformers jiwer faster_whisper

Note: you may need to restart the kernel to use updated packages.


In [32]:
import requests
import boto3
import uuid
import time
import string
import jiwer
import torch
import re
import os
from faster_whisper import WhisperModel


AUDIO_FILE = "HaTankistiot_n12-mp3.mp3"
MODEL_LIST = [
    ("Whisper v2", "large-v2"),
    ("Whisper v3", "large-v3"),
    ("Whisper Ivrit v2-d3-e3", "ivrit-ai/faster-whisper-v2-d3-e3"),
    ("Whisper Ivrit v2-d4", "ivrit-ai/faster-whisper-v2-d4"),
]

BEAM_SIZE = 5


def faster_whisper_transcribe(file, model_id):
    start_time = time.time()
    print(f"Transcribing using: {model_id}")
    result = ""
    model = WhisperModel(model_id, device="cuda", compute_type="float16")

    segs, _ = model.transcribe(file, language='he', beam_size=BEAM_SIZE)
    for s in segs:
        result += s.text

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Finished transcribing, execution time was: {execution_time:.3f} seconds.\n")
    print(f"{result}\n")

    return result


def amazon_transcribe(file_uri):
    # Initialize boto3 clients
    start_time = time.time()
    transcribe_client = boto3.client("transcribe")
    s3_client = boto3.client("s3")
    bucket_name = "virginia-transcribe"
    object_key = file_uri
    s3_client.upload_file(file_uri, bucket_name, object_key)

    job_name = str(uuid.uuid4())
    media = file_uri.split(".")[-1]
    file_uri = f's3://{bucket_name}/{object_key}'

    transcribe_client.start_transcription_job(
        TranscriptionJobName=job_name,
        Media={"MediaFileUri": file_uri},
        MediaFormat=media,
        LanguageCode="he-IL",
    )

    max_tries = 60
    while max_tries > 0:
        max_tries -= 1
        job = transcribe_client.get_transcription_job(TranscriptionJobName=job_name)
        job_status = job["TranscriptionJob"]["TranscriptionJobStatus"]
        if job_status in ["COMPLETED", "FAILED"]:
            print(f"Job {job_name} is {job_status}.")
            if job_status == "COMPLETED":
                break;
        else:
            print(f"Waiting for {job_name}. Current status is {job_status}.")
        time.sleep(10)

    json_url = job['TranscriptionJob']['Transcript']['TranscriptFileUri']
    file_content = requests.get(json_url)
    json_data = file_content.json()

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Finished transcribing, execution time was: {execution_time:.3f} seconds.\n")

    return json_data['results']['transcripts'][0]['transcript']


def remove_punctuation(text):
    # but maps the hyphen ('-') to a space (' ')
    translator = str.maketrans('', '', string.punctuation.replace('-', ''))
    translator[ord('-')] = ' '  # Map the hyphen to a space

    # Remove punctuation and replace hyphens with spaces
    text = text.translate(translator)

    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)

    return text


def WER_check(test, reference, model_name):
    error_transcribe = jiwer.process_words(reference, test)
    print(f"""############### {model_name} ###############
    WER (Word Error Rate) [lower is better] : {error_transcribe.wer * 100:.2f}%
    WIL (Word Information Lost) [lower is better] : {error_transcribe.wil * 100:.2f}%
    """)

In [33]:
asr_results = {model[0]: {'vanilla': faster_whisper_transcribe(AUDIO_FILE, model[1])} for model in MODEL_LIST}

Transcribing using: large-v2
Finished transcribing, execution time was: 44.150 seconds.

 הגר, הילה, תל שרה, מיכל, קרני, אופיר ותמר רק בנות 20, פלוס מינוס אבל הטנקיסטיות האלו והחברות שלהם לפלוגה עשו בשבעה באוקטובר היסטוריה האנשים הראשונות בישראל, כנראה בעולם כולו שלחמו בקרב שיריון קרב הרוי שבו הרגו עשרות מחבלים והצילו קיבוץ שלם בצילום הזה שמתפרסם כאן לראשונה רואים את הטנק של קרני, מפקד את הפלוגה פורץ בשערה את השערה של קיבוץ חולית בדרכו לטהר את הקיבוץ ממחבלים יום הקרב שלהם מתחיל בבוקר של שמחת תורה שבעה באוקטובר כשהמחבלים פורצים את הגדר מאזה לאוטף הטנקיסטיות נמצאות בכלל הרחק משם בגזרה אחרת, בגבול מצרים בבסיס שלהם, בניצנה הן מתחילות לנסוע עם שני כלים טנק וקטלן, סוג של רכב משוריין לכיוון צפון הטנק דוהר באופן מאוד יוצא דופן עשרות קילומטרים על כביש 232 בדרך לישובי אוטף עזה ליד מוצב סופה הן נתקלות בעשרות מחבלים ששועטים לעברן ומתקדמות לכיוון הגדר בגדר המערכת של רצועת עזה הן מבחינות באותו פתח ענק שיצרו המחבלים ומשאירות שם טנק אחד לשמור הטנק של מפקדת המחלקה שקד שמונע מעכשיו מאזתים להמשיך ולעבור 

In [34]:
asr_results.setdefault('Amazon Transcribe', {})['vanilla'] = amazon_transcribe(AUDIO_FILE)
print(f"############### Amazon Transcribe ###############\n{asr_results['Amazon Transcribe']['vanilla']}\n")

Waiting for 8c0c9f19-1165-4d21-9549-4129bf6dde63. Current status is IN_PROGRESS.
Waiting for 8c0c9f19-1165-4d21-9549-4129bf6dde63. Current status is IN_PROGRESS.
Waiting for 8c0c9f19-1165-4d21-9549-4129bf6dde63. Current status is IN_PROGRESS.
Waiting for 8c0c9f19-1165-4d21-9549-4129bf6dde63. Current status is IN_PROGRESS.
Job 8c0c9f19-1165-4d21-9549-4129bf6dde63 is COMPLETED.
Finished transcribing, execution time was: 40.674 seconds.

############### Amazon Transcribe ###############
להן הגר, הילה, טל שרה, מיכל, קרני, אופיר ותמר. רק בנות עשרים פלוס-מינוס, אבל הטנקיסטיות האלו והחברות שלהן לפלוגה, עשו ב שבע באוקטובר היסטוריה. הנשים הראשונות בישראל, כנראה בעולם כולו, שלחנו קמו בקרב שריון, קרב הרואי שבו הרגו עשרות מחבלים והצילו קיבוץ שלם. בצילום הזה, שמתפרסם כאן לראשונה, רואים את הטנק של קרני, מפקד את הפלוגה, פורץ בשערה את השער של קיבוץ חולית בדרכו לטהר את הקיבוץ במחבלים. יום הקרב שלהן מתחיל בבוקר של שמחת תורה, שבע באוקטובר. כשהמחבלים פורצים את הגדר מעזה לעוטף, הטנקיסטיות נמצאות בכלל הרחק 

In [35]:
asr_results.setdefault('Reference', {})['vanilla'] = open("HaTankistiot_n12_Yair.txt", "r").read()

In [36]:
def clean_numbers(text):
    number_map = {
        "20": "עשרים",
        "232": "שתיים שלוש שתיים",
        "-50": "חמישים",
        "-17": "שבע עשרה",
        "17": "שבע עשרה",
    }

    for number, word in number_map.items():
        text = text.replace(number, word)

    return text


for key, value in asr_results.items():
    asr_results[key]['clean_numbers'] = clean_numbers(value['vanilla'])
    print(f"########### {key} ###########\n{asr_results[key]['clean_numbers']}\n")

########### Whisper v2 ###########
 הגר, הילה, תל שרה, מיכל, קרני, אופיר ותמר רק בנות עשרים, פלוס מינוס אבל הטנקיסטיות האלו והחברות שלהם לפלוגה עשו בשבעה באוקטובר היסטוריה האנשים הראשונות בישראל, כנראה בעולם כולו שלחמו בקרב שיריון קרב הרוי שבו הרגו עשרות מחבלים והצילו קיבוץ שלם בצילום הזה שמתפרסם כאן לראשונה רואים את הטנק של קרני, מפקד את הפלוגה פורץ בשערה את השערה של קיבוץ חולית בדרכו לטהר את הקיבוץ ממחבלים יום הקרב שלהם מתחיל בבוקר של שמחת תורה שבעה באוקטובר כשהמחבלים פורצים את הגדר מאזה לאוטף הטנקיסטיות נמצאות בכלל הרחק משם בגזרה אחרת, בגבול מצרים בבסיס שלהם, בניצנה הן מתחילות לנסוע עם שני כלים טנק וקטלן, סוג של רכב משוריין לכיוון צפון הטנק דוהר באופן מאוד יוצא דופן עשרות קילומטרים על כביש שתיים שלוש שתיים בדרך לישובי אוטף עזה ליד מוצב סופה הן נתקלות בעשרות מחבלים ששועטים לעברן ומתקדמות לכיוון הגדר בגדר המערכת של רצועת עזה הן מבחינות באותו פתח ענק שיצרו המחבלים ומשאירות שם טנק אחד לשמור הטנק של מפקדת המחלקה שקד שמונע מעכשיו מאזתים להמשיך ולעבור דרכו לכיוון עזה או לכיוון ישראל שאר הל

In [37]:
for key, value in asr_results.items():
    asr_results[key]['rem_punct'] = remove_punctuation(value['clean_numbers'])
    print(f"########### {key} ###########\n{asr_results[key]['rem_punct']}\n")

########### Whisper v2 ###########
 הגר הילה תל שרה מיכל קרני אופיר ותמר רק בנות עשרים פלוס מינוס אבל הטנקיסטיות האלו והחברות שלהם לפלוגה עשו בשבעה באוקטובר היסטוריה האנשים הראשונות בישראל כנראה בעולם כולו שלחמו בקרב שיריון קרב הרוי שבו הרגו עשרות מחבלים והצילו קיבוץ שלם בצילום הזה שמתפרסם כאן לראשונה רואים את הטנק של קרני מפקד את הפלוגה פורץ בשערה את השערה של קיבוץ חולית בדרכו לטהר את הקיבוץ ממחבלים יום הקרב שלהם מתחיל בבוקר של שמחת תורה שבעה באוקטובר כשהמחבלים פורצים את הגדר מאזה לאוטף הטנקיסטיות נמצאות בכלל הרחק משם בגזרה אחרת בגבול מצרים בבסיס שלהם בניצנה הן מתחילות לנסוע עם שני כלים טנק וקטלן סוג של רכב משוריין לכיוון צפון הטנק דוהר באופן מאוד יוצא דופן עשרות קילומטרים על כביש שתיים שלוש שתיים בדרך לישובי אוטף עזה ליד מוצב סופה הן נתקלות בעשרות מחבלים ששועטים לעברן ומתקדמות לכיוון הגדר בגדר המערכת של רצועת עזה הן מבחינות באותו פתח ענק שיצרו המחבלים ומשאירות שם טנק אחד לשמור הטנק של מפקדת המחלקה שקד שמונע מעכשיו מאזתים להמשיך ולעבור דרכו לכיוון עזה או לכיוון ישראל שאר הלוחמות ממשיכ

In [38]:
for key, value in asr_results.items():
    if key != 'Reference':
        WER_check(test=value['rem_punct'], reference=asr_results['Reference']['rem_punct'], model_name=key)

############### Whisper v2 ###############
    WER (Word Error Rate) [lower is better] : 16.53%
    WIL (Word Information Lost) [lower is better] : 27.10%
    
############### Whisper v3 ###############
    WER (Word Error Rate) [lower is better] : 14.19%
    WIL (Word Information Lost) [lower is better] : 24.24%
    
############### Whisper Ivrit v2-d3-e3 ###############
    WER (Word Error Rate) [lower is better] : 14.02%
    WIL (Word Information Lost) [lower is better] : 23.43%
    
############### Whisper Ivrit v2-d4 ###############
    WER (Word Error Rate) [lower is better] : 10.35%
    WIL (Word Information Lost) [lower is better] : 17.35%
    
############### Amazon Transcribe ###############
    WER (Word Error Rate) [lower is better] : 9.85%
    WIL (Word Information Lost) [lower is better] : 16.86%
    
